## Load Data

In [3]:
function loadSequence(filename::String)
    rawData = readlines(filename)
    bootSeq = Array{Int64}(undef,7,length(rawData))
    for (i,line) in enumerate(rawData)
        state,field = split(line)
        bootSeq[1,i] = (state == "on") ? 1 : 0
        x,y,z = split(field,",")
        bootSeq[2:3,i] = parse.(Int64,split(strip(x,['x','y','z','=']),".."))
        bootSeq[4:5,i] = parse.(Int64,split(strip(y,['x','y','z','=']),".."))
        bootSeq[6:7,i] = parse.(Int64,split(strip(z,['x','y','z','=']),".."))
    end
    return bootSeq
end

loadSequence (generic function with 1 method)

In [4]:
testSeq = loadSequence("testInitializeSequence.txt")
testSeqP2 = loadSequence("testRebootSequence.txt")

7×60 Array{Int64,2}:
   1    1    1    1   0    1    0    1  …       0       0        1        0
  -5  -44  -49  -20  26  -41  -43  -33     -27365  -70369   -53470   -93533
  47    5   -1   34  39    5  -33   15      46395  -16548    21291    -4276
 -31  -27  -11  -40  40  -41  -45  -32      31009   22648  -120233   -16170
  22   21   42    6  50    6  -28   19      98017   78696   -33476    68771
 -19  -14  -10  -44  -2  -36    7  -34  …   15428   -1892   -44150  -104985
  33   35   38    1  11    8   25   11      76570   86821    38147   -24507

In [5]:
sequence = loadSequence("rebootSequence.txt");

In [6]:
size(sequence)

(7, 420)

In [7]:
testSeqSmall=[1 10 12 10 12 10 12;
1 11 13 11 13 11 13;
0 9 11 9 11 9 11;
1 10 10 10 10 10 10]
testSeqSmall = transpose(testSeqSmall)

7×4 LinearAlgebra.Transpose{Int64,Array{Int64,2}}:
  1   1   0   1
 10  11   9  10
 12  13  11  10
 10  11   9  10
 12  13  11  10
 10  11   9  10
 12  13  11  10

## Part 1

In [81]:
function initializeReactor(sequence::AbstractArray{Int64,2},bounds::AbstractArray{Int64,1})
    lx = bounds[2]-bounds[1]+1
    ly = bounds[4]-bounds[3]+1
    lz = bounds[6]-bounds[5]+1
    reactor = zeros(Int64,lx,ly,lz)
    i = Array{Int64}(undef,6)
    for step in eachcol(sequence)
        #ignore out-of-bounds directions
        if (any(step[2:2:end].<bounds[1:2:end]) || any(step[3:2:end].>bounds[2:2:end]))
            continue
        end
        
        #shift index
        i[1:2] = step[2:3] .- (bounds[1]-1)
        i[3:4] = step[4:5] .- (bounds[3]-1)
        i[5:6] = step[6:7] .- (bounds[5]-1)
        
        #assign state
        reactor[i[1]:i[2],i[3]:i[4],i[5]:i[6]] .= step[1]     
    end
    
    return reactor
end

initializeReactor (generic function with 1 method)

In [67]:
bounds = [5,15,5,15,5,15]
testReactor = initializeReactor(testSeqSmall,bounds);

In [72]:
sum(testReactor)

39

In [73]:
bounds = [-50,50,-50,50,-50,50]
testReactor = initializeReactor(testSeq,bounds);

In [74]:
sum(testReactor)

590784

In [75]:
bounds = [-50,50,-50,50,-50,50]
reactor = initializeReactor(sequence,bounds);
sum(reactor)

547648

## Part 2

In [52]:
function addCube(matrix::Array{Array{T,1},1},cube::AbstractArray{T,1}) where T<:Integer
    
    newMatrix = Array{Array{Int,1}}(undef,0)
    
    for mcube in matrix      
        #check if new cube overlaps with previous cube
        xbegin = max.(mcube[1:2:5],cube[1:2:5])
        xend = min.(mcube[2:2:6],cube[2:2:6])
        dx = xend .- xbegin

        #if cubes overlap
        if !any(dx.<0)
            if xbegin == cube[1:2:5] && xend == cube[2:2:6] 
                #if new cube is fully contained, return immediately without change to matrix
                return matrix
            elseif  xbegin == mcube[1:2:5] && xend == mcube[2:2:6]
                #if old cube is fully contained, do nothing 
            else #if partial overlap, split mcube along overlap lines in each dimension
                ranges = Array{Array{Int,1}}(undef,3,3) # i=direction, j=split
                for i in 0:2
                    if xbegin[1+i] > mcube[1+2*i] && xend[1+i] < mcube[2+2*i]
                        ranges[1+i,1] = [mcube[1+2*i],xbegin[1+i]-1]
                        ranges[1+i,2] = [xbegin[1+i],xend[1+i]]
                        ranges[1+i,3] = [xend[1+i]+1,mcube[2+2*i]]
                    elseif xbegin[1+i] > mcube[1+2*i]
                        ranges[1+i,1] = [mcube[1+2*i],xbegin[1+i]-1]
                        ranges[1+i,2] = [xbegin[1+i],xend[1+i]]
                    else
                        ranges[1+i,1] = [xbegin[1+i],xend[1+i]]
                        ranges[1+i,2] = [xend[1+i]+1,mcube[2+2*i]]
                    end
                end
                cubes = Array[]
                overlapBounds = Array{Int}(undef,2,3)
                overlapBounds[1,:]=xbegin[:]
                overlapBounds[2,:]=xend[:]
                overlapBounds = [overlapBounds...]
                for i in 1:3,j in 1:3,k in 1:3
                    if (isassigned(ranges,1,i) && isassigned(ranges,2,j) && isassigned(ranges,3,k))
                        push!(cubes,[ranges[1,i][1],ranges[1,i][2],ranges[2,j][1],ranges[2,j][2],ranges[3,k][1],ranges[3,k][2]])
                    end
                    if all(cubes[end] .== overlapBounds) #if cube is overlap cube delete it
                        pop!(cubes)
                    end
                end
                append!(newMatrix,cubes) #append new cubes
                
            end
        else #add mcube to newMatrix map
            push!(newMatrix,mcube)
        end
        
    end
    #add new cube to matrix
    push!(newMatrix,cube)
    
    return newMatrix
    
end

function subtractCube(matrix::Array{Array{T,1},1},cube::AbstractArray{T,1}) where T<:Integer
    newMatrix = Array{Array{Int,1}}(undef,0)
    
    for mcube in matrix
        #check for overlap
        xbegin = max.(mcube[1:2:5],cube[1:2:5]) #indices 1,3,5
        xend = min.(mcube[2:2:6],cube[2:2:6]) #indices 2,4,6
        dx = xend .- xbegin
        
        if !any(dx.<0) #if overlap, split mcube
            if  xbegin == mcube[1:2:5] && xend == mcube[2:2:6]
                #if old cube is fully contained, do nothing 
                
            else #if partial overlap, split mcube along overlap lines in each dimension
                ranges = Array{Array{Int,1}}(undef,3,3) # i=direction, j=split
                for i in 0:2
                    if xbegin[1+i] > mcube[1+2*i] && xend[1+i] < mcube[2+2*i]
                        ranges[1+i,1] = [mcube[1+2*i],xbegin[1+i]-1]
                        ranges[1+i,2] = [xbegin[1+i],xend[1+i]]
                        ranges[1+i,3] = [xend[1+i]+1,mcube[2+2*i]]
                    elseif xbegin[1+i] > mcube[1+2*i]
                        ranges[1+i,1] = [mcube[1+2*i],xbegin[1+i]-1]
                        ranges[1+i,2] = [xbegin[1+i],xend[1+i]]
                    else
                        ranges[1+i,1] = [xbegin[1+i],xend[1+i]]
                        ranges[1+i,2] = [xend[1+i]+1,mcube[2+2*i]]
                    end
                end
                cubes = Array[]
                overlapBounds = Array{Int}(undef,2,3)
                overlapBounds[1,:]=xbegin[:]
                overlapBounds[2,:]=xend[:]
                overlapBounds = [overlapBounds...]
                for i in 1:3,j in 1:3,k in 1:3
                    if (isassigned(ranges,1,i) && isassigned(ranges,2,j) && isassigned(ranges,3,k))
                        push!(cubes,[ranges[1,i][1],ranges[1,i][2],ranges[2,j][1],ranges[2,j][2],ranges[3,k][1],ranges[3,k][2]])
                    end
                    if all(cubes[end] .== overlapBounds) #if cube is overlap cube delete it
                        pop!(cubes)
                    end
                end
                append!(newMatrix,cubes) #append new cubes
                
            end
        else #add mcube to newMatrix map
            push!(newMatrix,mcube)
        end
    end
    
    return newMatrix
end

function countCubes(matrix::Array{Array{T,1},1}) where T<:Integer
    numOnCubes = 0
    for mcube in matrix
        numOnCubes += prod(mcube[2:2:6]-mcube[1:2:5].+1)
    end
    return numOnCubes
end

function rebootReactor(sequence::AbstractArray{Int64,2})
    allCubes = Array{Array{Int,1}}(undef,0)
    for step in eachcol(sequence)
        if step[1] == 1
            allCubes=addCube(allCubes,step[2:end])
        else
            allCubes=subtractCube(allCubes,step[2:end])
        end
        #println(countCubes(allCubes))#,"\n",allCubes)
    end
    numOnCubes = countCubes(allCubes)
    return numOnCubes#,allCubes
end

rebootReactor (generic function with 1 method)

In [50]:
rebootReactor(testSeqSmall)

27
46
38
39


39

In [53]:
2758514936282235 == rebootReactor(testSeqP2)

true

In [57]:
@time rebootReactor(sequence)

  4.599743 seconds (53.44 M allocations: 5.117 GiB, 9.74% gc time)


1206644425246111